In [1]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv
Saving test.csv to test.csv


In [2]:
import pandas as pd
data=pd.read_csv("train.csv")
data.head(10)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,NaN,Ever wonder how Britain’s most iconic pop pian...,1
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
8,8,Excerpts From a Draft Script for Donald Trump’...,NaN,Donald J. Trump is scheduled to make a highly ...,0
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0


In [3]:
data=data.dropna()
data.reset_index(inplace=True)

In [4]:
X = data.drop("label",axis=1)
y = data["label"]

In [5]:
print(X.shape)
print(y.shape)

(18285, 5)
(18285,)


In [6]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

In [7]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
messages=X.copy()
ps = PorterStemmer()
corpus = []
for i in range(len(messages)):
  review = re.sub("[^a-zA-Z]"," ",messages['title'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [9]:
vocab_size=5000
onehot_repr=[one_hot(words,vocab_size)for words in corpus]


In [10]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2502  693 4625]
 [   0    0    0 ... 4322 4268 3827]
 [   0    0    0 ... 1086 3934  148]
 ...
 [   0    0    0 ... 3762 2589 2820]
 [   0    0    0 ... 2092  542 2253]
 [   0    0    0 ... 2485  172  427]]


In [11]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(vocab_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [12]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [14]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)


Epoch 1/10
192/192 [==============================] - 13s 46ms/step - loss: 0.3387 - accuracy: 0.8389 - val_loss: 0.1928 - val_accuracy: 0.9185
Epoch 2/10
192/192 [==============================] - 11s 57ms/step - loss: 0.1411 - accuracy: 0.9456 - val_loss: 0.1872 - val_accuracy: 0.9229
Epoch 3/10
192/192 [==============================] - 14s 74ms/step - loss: 0.0966 - accuracy: 0.9637 - val_loss: 0.2101 - val_accuracy: 0.9190
Epoch 4/10
192/192 [==============================] - 12s 60ms/step - loss: 0.0684 - accuracy: 0.9767 - val_loss: 0.2446 - val_accuracy: 0.9170
Epoch 5/10
192/192 [==============================] - 8s 44ms/step - loss: 0.0441 - accuracy: 0.9861 - val_loss: 0.3260 - val_accuracy: 0.9173
Epoch 6/10
192/192 [==============================] - 9s 49ms/step - loss: 0.0247 - accuracy: 0.9927 - val_loss: 0.3690 - val_accuracy: 0.9165
Epoch 7/10
192/192 [==============================] - 10s 51ms/step - loss: 0.0183 - accuracy: 0.9944 - val_loss: 0.4189 - val_accuracy: 0

In [15]:
y_pred = model.predict(X_test)
y_pred = np.round(y_pred).astype(int)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

189/189 [==============================] - 2s 8ms/step


array([[3086,  333],
       [ 175, 2441]])

In [16]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)


0.915824357912179